<a href="https://colab.research.google.com/github/boraboyacioglu-itu/voice_bot/blob/main/tf_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import necessary libraries.

In [1]:
import numpy as np
import tensorflow as tf

import json
import pickle

from sklearn.preprocessing import LabelEncoder

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Open and read the knowledge base.
with open('intents.json') as f:
    data = json.load(f)

# Initialise the arrays.
training_sentences = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

# Number of different topics.
num_classes = len(labels)

In [3]:
# Initialise and fit the encoder to the training labels.
label_encoder = LabelEncoder()
label_encoder.fit(training_labels)
training_labels = label_encoder.transform(training_labels)

In [4]:
# Define the parameters.
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

# Tokenise the sentences.
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [5]:
# Build the model.
model = Sequential()

# Add layers.
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 9)                 153       
                                                                 
Total params: 16697 (65.22 KB)
Trainable params: 16697 (65.22 KB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [6]:
# Fit the model.
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
2/2 [==============================] - 3s 14ms/step - loss: 2.1958 - accuracy: 0.1316
Epoch 2/500
2/2 [==============================] - 0s 23ms/step - loss: 2.1949 - accuracy: 0.1316
Epoch 3/500
2/2 [==============================] - 0s 19ms/step - loss: 2.1940 - accuracy: 0.1316
Epoch 4/500
2/2 [==============================] - 0s 15ms/step - loss: 2.1934 - accuracy: 0.1316
Epoch 5/500
2/2 [==============================] - 0s 19ms/step - loss: 2.1927 - accuracy: 0.1316
Epoch 6/500
2/2 [==============================] - 0s 16ms/step - loss: 2.1919 - accuracy: 0.1316
Epoch 7/500
2/2 [==============================] - 0s 12ms/step - loss: 2.1912 - accuracy: 0.1316
Epoch 8/500
2/2 [==============================] - 0s 13ms/step - loss: 2.1906 - accuracy: 0.1316
Epoch 9/500
2/2 [==============================] - 0s 21ms/step - loss: 2.1901 - accuracy: 0.1316
Epoch 10/500
2/2 [==============================] - 0s 10ms/step - loss: 2.1896 - accuracy: 0.1316
Epoch 11/500
2/2 [=

In [7]:
# Data locations.
data_loc = 'data/'

# Save the trained model.
model.save(data_loc + 'chat_model')

# Save the fitted tokenizer.
with open(data_loc + 'tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the fitted label encoder.
with open(data_loc + 'label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(label_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
import json
import numpy as np

import random
import pickle

from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

with open("intents.json") as file:
    data = json.load(file)

# Define the colours for terminal output.
cr = "\033[91m"
cg = "\033[92m"
cb = "\033[94m"
ss = "\033[0m"

# Data locations.
data_loc = 'data/'

def chat():
    # Load trained model.
    model = keras.models.load_model(data_loc + 'chat_model')

    # Load tokenizer object.
    with open(data_loc + 'tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # Load label encoder object.
    with open(data_loc + 'label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # Parameters.
    max_len = 20

    while True:
        print(f"{cr}User: {ss}", end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]), truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(f"{cb}ChatBot: {ss}", np.random.choice(i['responses']))

print(f"{cb}Start messaging with the bot. Type 'quit' to stop.{ss}")
chat()

Start messaging with the bot (type quit to stop)!
User: mourning
1/1 [==============================] - 0s 73ms/step
ChatBot:  Hi
User: quit
